# Running the repo

You might have to add a HF token. Please don't steal my token!

If you want to edit any of the .py files, like benchmark.py or prompts.py, you can edit them directly in colab (Files [on the left bar] -> My Drive -> ....), but then you have to restart the runtime. (runtime -> restart session). Note that you don't have to do the pip install cell afterwards, since the runtime stil exists, it just got reset. Only if you disconnect/delete the runtime (such as if you turn your computer off and come back a few hours later) will you have to re-run the pip install cell.

In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
HF_TOKEN = os.environ.get("HF_TOKEN")
# Paste your token inside the quotes
login(HF_TOKEN)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import importlib
import benchmark
import llm_agent
from llm_providers import create_llm
importlib.reload(benchmark)
importlib.reload(llm_agent)

<module 'llm_agent' from '/Users/jooseunglee/Desktop/codenames_llm/llm_agent.py'>

## Model and Persona Configuration

This cell sets up the LLM model and optional personas for both codemaster and guesser agents.

**To switch models:** Simply change `MODEL_CHOICE` to one of:
- `"llama31"` - LLaMA 3.1 8B Instruct (default)
- `"qwen3"` - Qwen 2.5 7B Instruct
- `"mistral"` - Mistral 7B Instruct v0.2

**To use personas:** Set `CODEMASTER_PERSONA` and `GUESSER_PERSONA` to persona IDs ("1" through "20")
- Use `None` for no persona (baseline)
- See personas.json for full persona descriptions
- Same persona space applies to both roles

**Key features:**
- Uses proper chat templates for each model (automatically detected)
- Supports 4-bit quantization to save memory
- Shared LLM instance for both agents (memory efficient)
- Persona injection follows NeurIPS 2024 trust game approach

In [ ]:
import llm_agent
from llm_providers import create_llm
from persona_loader import list_persona_ids

# ===== MODEL CONFIGURATION =====
# Choose which model to use by setting MODEL_CHOICE
# Options: "llama31", "qwen3", "mistral"

MODEL_CHOICE = "llama31"  # Change this to switch models

MODEL_CONFIGS = {
    "llama31": {
        "type": "local_hf",
        "model_name": "meta-llama/Llama-3.1-8B-Instruct",
        "temperature": 0.6,
        "max_tokens": 1024,
        # "load_in_4bit": True
    },
    "qwen3": {
        "type": "local_hf",
        "model_name": "Qwen/Qwen3-8B",
        "temperature": 0.7,
        "max_tokens": 2048,  # 10x increase from default 200
        "disable_reasoning": True,  # Suppress <think> tags
    },
    "mistral": {
        "type": "local_hf",
        "model_name": "mistralai/Mistral-7B-Instruct-v0.2",
        "temperature": 0.7,
        # "load_in_4bit": True
    }
}

# ===== PERSONA CONFIGURATION =====
# Set persona IDs for codemaster and guesser (use None for no persona)
# Available persona IDs: "1" through "20" (see personas.json for details)
#
# Examples of the 4 combinations:
# 1. No personas (baseline):
#    codemaster_persona, guesser_persona = None, None
#
# 2. Only codemaster has persona:
#    codemaster_persona, guesser_persona = "1", None
#
# 3. Only guesser has persona:
#    codemaster_persona, guesser_persona = None, "2"
#
# 4. Both have personas (can be same or different):
#    codemaster_persona, guesser_persona = "1", "3"

codemaster_persona = "1"  # Change to "1", "2", etc. to enable persona
guesser_persona = "2"     # Change to "1", "2", etc. to enable persona

# ===== PERSONA SHARING TOGGLE =====
# When True: agents know each other's personas and can adapt their communication
# When False (default): personas remain isolated, agents unaware of partner's background
shared_persona = False  # Change to True to enable persona sharing

# List all available personas
print("Available personas:", ", ".join(list_persona_ids()))

# Get the selected config
config = MODEL_CONFIGS[MODEL_CHOICE]
print(f"Loading model: {config['model_name']}")
print(f"Codemaster Persona: {codemaster_persona or 'None (baseline)'}")
print(f"Guesser Persona: {guesser_persona or 'None (baseline)'}")
print(f"Persona Sharing: {'Enabled' if shared_persona else 'Disabled'}")

# Create shared LLM instance
shared_llm_instance = create_llm(config)

# Create agents with shared instance and optional personas
codemaster = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
codemaster.initialize_role('codemaster',
                          persona_id=codemaster_persona,
                          partner_persona_id=guesser_persona,
                          shared_persona=shared_persona)

guesser = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
guesser.initialize_role('guesser',
                       persona_id=guesser_persona,
                       partner_persona_id=codemaster_persona,
                       shared_persona=shared_persona)

print(f"✓ Codemaster and Guesser initialized with {MODEL_CHOICE}")

In [6]:
bnch = benchmark.CodeNamesBenchmark()

# Run collaborative game with current model
print(f"\n{'='*60}")
print(f"Running game with {MODEL_CHOICE}: {config['model_name']}")
print(f"{'='*60}\n")

results = bnch.run_collab_matchup(config, num_games=1, codemaster=codemaster, guesser=guesser)

print(f"\n{'='*60}")
print(f"Results for {MODEL_CHOICE}")
print(f"{'='*60}")
print(results)

# Optionally save logs
# import os
# os.makedirs("logs", exist_ok=True)
# benchmark.export_turn_logs(f"logs/codenames_turns_{MODEL_CHOICE}.json")
# print(f"\nTurn logs saved to logs/codenames_turns_{MODEL_CHOICE}.json")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Running game with llama31: meta-llama/Llama-3.1-8B-Instruct


Starting collaborative game 0

Board State:
MATCH | SPACE | WATER | PLASTIC | ROOT
SCALE | JACK | CANADA | TORCH | COMIC
CELL | QUEEN | CARD | PASTE | LONDON
LAP | BUFFALO | SPIKE | CHAIR | FORK
ROBIN | FIELD | NOTE | CRICKET | POOL


=== Turn 1 ===
Remaining words to guess: PLASTIC, ROBIN, LONDON, CARD, COMIC, FORK, PASTE, FIELD, ROOT


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```json
{
  "reasoning": "The clue I've chosen is a word that has multiple connections to the given words, while avoiding any direct references to the neutral words or the assassin word. I'm targeting the concept of 'utensil' or 'tool' as it relates to 'FORK', 'CARD' and 'PASTE'.",
  "clue": "utensil",
  "number": 3
}
Codemaster's clue: utensil 3
(Reasoning: The clue I've chosen is a word that has multiple connections to the given words, while avoiding any direct references to the neutral words or the assassin word. I'm targeting the concept of 'utensil' or 'tool' as it relates to 'FORK', 'CARD' and 'PASTE'.)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

codemaster raw output: ```json
{
  "reasoning": "Considering the recent clue 'paper 2' and the presence of 'CARD', I'm looking for a connection between 'PASTE' and 'PAPER'. Since 'PASTE' is often associated with 'PAPER', I'm hoping to link it to the existing 'paper 2' clue.",
  "clue": "paste",
  "number": 2
}
**CRITICAL FAILURE**: Codemaster failed validation or parsing: Clue word 'PASTE' is on the board.. Raw output: '```json\n{\n  "reasoning": "Considering the recent clue \'paper 2\' and the presence of \'CARD\', I\'m looking for a connection between \'PASTE\' and \'PAPER\'. Since \'PASTE\' is often associated with \'PAPER\', I\'m hoping to link it to the existing \'paper 2\' clue.",\n  "clue": "paste",\n  "number": 2\n}'
Codemaster's clue: FAIL_NO_WORD 0
(Reasoning: Parsing/Validation Error: Clue word 'PASTE' is on the board.)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


guesser raw output: ```json
{
  "reasoning": "The clue is 'FAIL_NO_WORD 0', which means there are 0 words related to 'FAIL_NO_WORD'. Given the previous clues 'utensil 3' and 'paper 2', which both had multiple related words, I'm going to take a chance and guess a word that is not related to the given clue. Since we already have 'FORK' and 'CARD', I'm going to choose a word that is not a utensil or a paper-related item. I'll pick a word that is often associated with a 'FOR' prefix, which is a common prefix in English words.",
  "guess": "FORK"
}
**WARNING**: Guesser output 'FORK' is not available on board. Forcing STOP.
Guesser's guess: STOP
(Reasoning: Guessed invalid word FORK. Forcing stop.)
Guessed a word not on the board

Board State:
MATCH | SPACE | WATER | PLASTIC | ROOT
SCALE | JACK | CANADA | TORCH | COMIC
CELL | QUEEN | [CARD] | PASTE | LONDON
LAP | BUFFALO | SPIKE | CHAIR | [FORK]
ROBIN | FIELD | [NOTE] | CRICKET | POOL


=== Turn 4 ===
Remaining words to guess: PLASTIC, ROBIN

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```json
{
  "reasoning": "Considering the words PLASTIC, PASTE, and COMIC, I'm connecting them to a common theme related to art and creative expression. A clue that relates to this theme could be 'ART' and has multiple connections to the words on the board.",
  "clue": "ART",
  "number": 3
}
Codemaster's clue: ART 3
(Reasoning: Considering the words PLASTIC, PASTE, and COMIC, I'm connecting them to a common theme related to art and creative expression. A clue that relates to this theme could be 'ART' and has multiple connections to the words on the board.)
guesser raw output: ```json
{
  "reasoning": "Considering the clue 'ART 3', I'm thinking of words that are related to art. Since there's a 'TORCH' on the board, I'm guessing it could be related to art, possibly as a tool or medium. Additionally, 'COMIC' is also a form of art, and 'SCALE' could be related to art in the sense of measurement or proportion.",
  "guess": "TORCH"
}
Guesser's guess: TORCH
(Reasoning: 

In [7]:
results['average_words_per_clue']

0.5